In [1]:
import pandas as pd

import os
import sys
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")

import requests
import time
import datetime as datetime

from bs4 import BeautifulSoup

from google.cloud import bigquery

from utils.scraping_and_api_utils import drop_dot_make_int

# Scraping Section

In [93]:
# Setting up the client
client = bigquery.Client.from_service_account_json(os.environ.get('JSON_KEY'))

# Getting the names of the projects and datasets
project = os.environ.get('PROJECT')
dataset = os.environ.get('DATASET')
links_table = os.environ.get('LINKS_TABLE')
game_data_table = os.environ.get('GAME_DATA_TABLE')
missed_data_table = os.environ.get('MISSED_DATA_TABLE')

# Creating the query and running it
query = f"""SELECT * FROM `legendary-game-recs.game_data_01_24.missed_data`"""
query_job = client.query(query)
results = query_job.result()

# Turning our Query into a DF
missed_data = results.to_dataframe()

In [6]:
frame = missed_data.drop_duplicates().copy()
frame.shape

In [8]:
# Setting count for montoring progress
# Only the first 116928 are relevant
count = 0

for game in frame['link']:
    # Monitoring progress
    if count % 10 == True:
        print(count)
    else:
        pass

    count += 1

    try:
        # Setting up the html parser + beautful soup
        game_response = requests.get(f"https://www.backloggd.com{game}")
        game_soup = BeautifulSoup(game_response.content, 'html.parser')


        # Scraping game_id
        game_id = game_soup.find('div', class_='card mx-auto game-cover overlay-hide')['game_id']


        # Scraping the title
        title = game_soup.find('h1', class_='mb-0').string

        # Scraping the date, and making it DateTime
        date_step = game_soup.find('div', class_='col-auto mt-auto pr-0')
        date_step2 = date_step.find('a', href=True).string
        if date_step2 == 'TBD':
            date_as_datetime = datetime.datetime(1, 1, 1).strftime("%Y-%m-%d")

        else:
            date_as_datetime = datetime.datetime.strptime(date_step2, '%b %d, %Y').strftime('%Y-%m-%d')


        # Getting the plays, playing, backlogs and wishlist information

        counter = game_soup.find('div', id='log-counters').find_all('a', class_='plays-counter')

        plays = drop_dot_make_int(counter[0].find('p', class_='mb-0').string.replace('K', '000'))
        playing = drop_dot_make_int(counter[1].find('p', class_='mb-0').string.replace('K', '000'))
        backlogs = drop_dot_make_int(counter[2].find('p', class_='mb-0').string.replace('K', '000'))
        wishlist = drop_dot_make_int(counter[3].find('p', class_='mb-0').string.replace('K', '000'))

        # Get a list of publis
        publisher_list = []
        try:
            publishers = game_soup.find('div', class_='col-auto pl-lg-1 sub-title').find_all('a', href=True)
            for i in range(len(publishers)):
                publisher_list.append(publishers[i].string)
        except:
            publisher_list = []

        # Get Average review score
        review_score = game_soup.find('h1', class_='text-center').string
        if review_score == 'N/A':
            review_score = 0
        else:
            review_score = float(review_score)

        # Get genres
        genres = game_soup.find_all('p', class_='genre-tag')
        genre_list = []
        for i in range(len(genres)):
            genre_list.append(genres[i].string)

        # Get platforms
        platforms = game_soup.find_all('a', class_='game-page-platform')
        platform_list = []
        for i in range(len(platforms)):
            platform_list.append(platforms[i].get_text(strip=True))

        # Get description
        description = game_soup.find('div', id='collapseSummary').get_text(strip=True)

        # Get number of reviews - number of lists associated is here as well
        lists_reviews = game_soup.find_all('p', class_='game-page-sidecard')

        total_lists = drop_dot_make_int(lists_reviews[0].get_text(strip=True).strip(" Lists").replace('K', '000'))
        total_reviews = drop_dot_make_int(lists_reviews[1].get_text(strip=True).strip(" Reviews").replace('K', '000'))

        # Get game category + main (If applicable)
        # If the search for category fails, then the game is the main game
        try:
            main_game = game_soup.find('p', class_='mb-2 game-parent-category').find('a').get_text()
            full_sentence = game_soup.find('p', class_='mb-2 game-parent-category').get_text()
            category = full_sentence.replace(main_game, '').strip()
        except:
            main_game = title
            category = 'main'

        # Get ratings, ten categories from 0.5 to 5.0

        ratings = game_soup.find_all('div', class_="col px-0 top-tooltip")

        ratings_zero_five = int(ratings[0]['data-tippy-content'].split(' |')[0])
        ratings_one_zero = int(ratings[1]['data-tippy-content'].split(' |')[0])
        ratings_one_five = int(ratings[2]['data-tippy-content'].split(' |')[0])
        ratings_two_zero = int(ratings[3]['data-tippy-content'].split(' |')[0])
        ratings_two_five = int(ratings[4]['data-tippy-content'].split(' |')[0])
        ratings_three_zero = int(ratings[5]['data-tippy-content'].split(' |')[0])
        ratings_three_five = int(ratings[6]['data-tippy-content'].split(' |')[0])
        ratings_four_zero = int(ratings[7]['data-tippy-content'].split(' |')[0])
        ratings_four_five = int(ratings[8]['data-tippy-content'].split(' |')[0])
        ratings_five_zero = int(ratings[9]['data-tippy-content'].split(' |')[0])

        # Creating a dictionary to insert into GCP
        data_dict = {'title': title,
                        'release_date': date_as_datetime,
                        'plays':plays,
                        'playing':playing,
                        'backlogs':backlogs,
                        'wishlist':wishlist,
                        'developers':str(publisher_list),
                        'avg_review':review_score,
                        'genres':str(genre_list),
                        'platforms':str(platform_list),
                        'description':description,
                        'total_reviews':total_reviews,
                        'total_lists':total_lists,
                        'category':category,
                        'main':main_game,
                        'ratings_zero_five':ratings_zero_five,
                        'ratings_one_zero':ratings_one_zero,
                        'ratings_one_five':ratings_one_five,
                        'ratings_two_zero':ratings_two_zero,
                        'ratings_two_five':ratings_two_five,
                        'ratings_three_zero':ratings_three_zero,
                        'ratings_three_five':ratings_three_five,
                        'ratings_four_zero':ratings_four_zero,
                        'ratings_four_five':ratings_four_five,
                        'ratings_five_zero':ratings_five_zero,
                        'url':game,
                        'game_id':game_id}

        # Inserting into GCP
        table = client.dataset(dataset).table('game_data_test')
        errors = client.insert_rows_json(table, [data_dict])

        # Relaying errors for some sexy debugging
        if errors:
            print(f'Failed {game}, {errors}')
            #Saving missed data, so we can try get it later
            table = client.dataset(dataset).table('missed_data_2nd')
            errors = client.insert_rows_json(table, [{'link':game}])
        else:
            print(f'all good {count}')


    except:
        # Relaying when Backlogged rejects us
        print(f'Page failure at {game}, count = {count}')
        #Saving missed data, so we can try get it later
        table = client.dataset(dataset).table('missed_data_2nd')
        errors = client.insert_rows_json(table, [{'link':game}])

        time.sleep(3)


all good 1
1
all good 2
all good 3
all good 4
all good 5
all good 6
all good 7
all good 8
all good 9
all good 10
all good 11
11
all good 12
all good 13
all good 14
all good 15
all good 16
all good 17
all good 18
all good 19
all good 20
all good 21
21
all good 22
all good 23
all good 24
all good 25
all good 26
all good 27
all good 28
all good 29
all good 30
all good 31
31
all good 32
all good 33
all good 34
all good 35
all good 36
all good 37
all good 38
all good 39
all good 40
all good 41
41
all good 42
all good 43
all good 44
all good 45
all good 46
all good 47
all good 48
all good 49
all good 50
all good 51
51
all good 52
all good 53
all good 54
all good 55
all good 56
all good 57
all good 58
all good 59
all good 60
all good 61
61
all good 62
all good 63
all good 64
all good 65
all good 66
all good 67
all good 68
all good 69
all good 70
all good 71
71
all good 72
all good 73
all good 74
all good 75
all good 76
all good 77
all good 78
all good 79
all good 80
all good 81
81
all good 82

# API Section

In [2]:
import os
import sys
import pandas as pd
import requests
from igdb.wrapper import IGDBWrapper
import json
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")

from google.cloud import bigquery
from google.cloud import secretmanager

from utils.processing_utils import process_raw_data
from utils.scraping_and_api_utils import prepare_json_df

from preprocessing.api_processing import *

import time

In [ ]:
#LEGACY CODE

# query_1 = f"""SELECT * FROM `legendary-game-recs.game_data_01_24.game_data`"""
# query_2 = f"""SELECT * FROM `legendary-game-recs.game_data_01_24.game_data_test`"""

# # query = f"""SELECT * FROM `{project}.{dataset}.{links_table}"""
# query_job_1 = client.query(query_1)
# results_1 = query_job_1.result()
# game_data_1 = results_1.to_dataframe()

# query_job_2 = client.query(query_2)
# results_2 = query_job_2.result()
# game_data_2 = results_2.to_dataframe()

# data = pd.concat([game_data_1, game_data_2]).reset_index(drop=True)

# re, up = process_raw_data(data, 2024, 1, 24)

# all_data = pd.concat([re,up])

In [15]:
# Create a Secret Manager client
secret_client = secretmanager.SecretManagerServiceClient()

# Define the name of the secret
secret_id_name = "projects/legendary-game-recs/secrets/CLIENT_ID/versions/latest"
secret_client_name = "projects/legendary-game-recs/secrets/CLIENT_SECRET/versions/latest"
secret_grant_name = "projects/legendary-game-recs/secrets/GRANT_TYPE/versions/latest"

# # Access the secret
secret_id_response = secret_client.access_secret_version(request={"name": secret_id_name})
secret_client_response = secret_client.access_secret_version(request={"name": secret_client_name})
secret_grant_response = secret_client.access_secret_version(request={"name": secret_grant_name})

# Credentials for calling the API
CLIENT_ID = secret_id_response.payload.data.decode("UTF-8")
CLIENT_SECRET = secret_client_response.payload.data.decode("UTF-8")
GRANT_TYPE = secret_grant_response.payload.data.decode("UTF-8")

In [16]:
# List of games used in the model
client = bigquery.Client.from_service_account_json(os.environ.get('JSON_KEY'))

project = os.environ.get('PROJECT')
dataset = os.environ.get('DATASET')
links_table = os.environ.get('LINKS_TABLE')
game_data_table = os.environ.get('GAME_DATA_TABLE')
missed_data_table = os.environ.get('MISSED_DATA_TABLE')

query = f"""
            SELECT DISTINCT * FROM `legendary-game-recs.game_data_01_24.game_data`
            WHERE release_date != '0001-01-01T00:00:00' AND category = 'main' AND avg_review > 0

            UNION ALL

            SELECT DISTINCT * FROM `legendary-game-recs.game_data_01_24.game_data_test`
            WHERE release_date != '0001-01-01T00:00:00' AND category = 'main' AND avg_review > 0
        """

query_job = client.query(query)
results = query_job.result()
all_data = results.to_dataframe()

list_of_game_id = all_data['game_id']

In [37]:
all_data.shapea1

(36316, 27)

In [35]:
# table_ref = client.dataset('game_data_01_24').table('all_data')
# tabless = client.get_table(table_ref)

# for field in tabless.schema:
#     print(f"{field.name}: {field.field_type},")

In [17]:
#Generating the access token
response = requests.post(f'https://id.twitch.tv/oauth2/token?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&grant_type={GRANT_TYPE}')
response_json = response.json()
ACCESS_TOKEN = response_json['access_token']


#This is a wrapper from IGDB just for their API
wrapper = IGDBWrapper(CLIENT_ID, ACCESS_TOKEN)

no_data = []
list_sons = []
count = 0

for game in list_of_game_id[:15]:
    count += 1

    try:
        the_feat = json.loads(wrapper.api_request('games',
                f'fields franchise, franchises, storyline, aggregated_rating,aggregated_rating_count, game_engines.name, game_modes.name, multiplayer_modes, player_perspectives.name, themes.name, rating; where id = {int(game)};'))
        list_sons.append(the_feat[0])
        print(f'Success at {count}')

    except:
        print(f'failed at {count}')
        no_data.append(game)


Success at 1
Success at 2
Success at 3
Success at 4
Success at 5
Success at 6
Success at 7
Success at 8
Success at 9
Success at 10
Success at 11
Success at 12
Success at 13
Success at 14
Success at 15


In [6]:
missed = pd.DataFrame(no_data, columns=['game_id'])

In [7]:
# # This is used for getting the information from age_ratings.
# def get_category_descriptions(json_dict):
#     category_list = []

#     if type(json_dict) == float:
#         category_list = None
#     else:
#         for i in range(len(json_dict)):
#             try:
#                 descriptions = json_dict[i]['content_descriptions']
#                 for i in range(len(descriptions)):
#                     rating_descript = descriptions[i]['category']
#                     category_list.append(rating_descript)

#             except:
#                 pass

#     return category_list

# # This goes through each key in the JSON and creates a list of items for the variable
# def get_names_from_dict(dict_list):
#     list_of_items = []
#     try:
#         for i in range(len(dict_list)):
#             item = dict_list[i]['name']
#             list_of_items.append(item)
#     except:
#         list_of_items = None

#     return list_of_items

# # This uses the above functions to create a final, clean df
# def prepare_json_df(original_json_df):
#     original_json_df['themes'] = original_json_df['themes'].apply(get_names_from_dict)
#     original_json_df['game_modes'] = original_json_df['game_modes'].apply(get_names_from_dict)
#     original_json_df['player_perspectives'] = original_json_df['player_perspectives'].apply(get_names_from_dict)
#     original_json_df['game_engines'] = original_json_df['game_engines'].apply(get_names_from_dict)
#     # original_json_df['age_ratings'] = original_json_df['age_ratings'].apply(get_category_descriptions)

#     return original_json_df

In [39]:
# Creates a df from the list of dictionaries
new_df = prepare_json_df(pd.DataFrame(list_sons))

In [42]:
everything = all_data.merge(new_df, right_on='id', left_on='game_id', how='right')

In [43]:
everything

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,aggregated_rating_count,game_engines,game_modes,player_perspectives,rating,storyline,themes,franchises,multiplayer_modes,franchise
0,Bard's Gold,2015-08-14,14,0,21,1,"['EastAsiaSoft', 'Erdem Sen']",2.0,"['Indie', 'Platform', 'RPG']","['Windows PC', 'Mac', 'Linux', 'PlayStation 4'...",...,3.0,[GameMaker: Studio],[Single player],[Side view],70.000000,"""While I was adventuring, I got my saddle cont...","[Action, Fantasy, Historical]",NaN,NaN,NaN
1,Mario Party 10,2015-03-12,2400,4,146,82,['Nintendo'],2.0,"['Card & Board Game', 'Quiz/Trivia']",['Wii U'],...,16.0,None,"[Single player, Multiplayer, Split screen]",[Third person],61.310368,NaN,"[Action, Party]",[845],[19938],NaN
2,Homefront,2011-03-15,2400,9,404,96,"['Digital Extremes', 'THQ']",2.0,['Shooter'],"['Windows PC', 'Xbox 360', 'PlayStation 3']",...,14.0,[Unreal Engine],"[Single player, Multiplayer]",[First person],68.646107,The year is 2027. The world has suffered a dec...,"[Action, Fantasy, Historical, Warfare]",NaN,NaN,NaN
3,Flappy Bird,2013-05-24,12000,15,97,17,['.GEARS Studios'],2.0,['Arcade'],"['Android', 'iOS']",...,NaN,None,[Single player],[Side view],47.128321,NaN,[Action],NaN,NaN,NaN
4,Saints Row,2022-08-23,1300,105,863,383,"['Volition', 'Deep Silver']",2.0,"['Adventure', 'Shooter']","['Windows PC', 'PlayStation 4', 'Xbox One', 'P...",...,7.0,None,"[Single player, Multiplayer, Co-operative]",[Third person],46.657578,"As the future Boss, with Neenah, Kevin, and El...","[Action, Open world]",NaN,[25622],NaN
5,Finding Nemo,2003-05-09,1300,5,85,33,"[""Traveller's Tales"", 'THQ']",2.0,['Adventure'],"['Nintendo GameCube', 'Xbox', 'PlayStation 2']",...,3.0,None,[Single player],[Third person],60.707378,NaN,"[Action, Educational]",[167],NaN,167.0
6,Realm Royale Reforged,2018-06-05,1200,5,71,3,['Hi-Rez Studios'],2.0,"['Adventure', 'Shooter']","['Windows PC', 'PlayStation 4', 'Xbox One', 'N...",...,2.0,None,"[Single player, Multiplayer, Battle Royale]","[First person, Third person]",60.268947,NaN,"[Action, Fantasy]",NaN,NaN,NaN
7,Fallout 76,2018-11-14,4600,182,1200,256,"['Bethesda Softworks', 'Bethesda Game Studios']",2.0,"['RPG', 'Shooter']","['Windows PC', 'PlayStation 4', 'Xbox One']",...,17.0,[Creation Engine],"[Multiplayer, Co-operative, Massively Multipla...","[First person, Third person]",57.717230,NaN,"[Action, Science fiction, Survival, Open world]",[1034],"[18544, 18545, 18546]",NaN
8,Super Mario Run,2016-12-15,4900,27,117,43,"['Nintendo', 'Nintendo EPD']",2.0,"['Adventure', 'Arcade', 'Platform']","['Android', 'iOS']",...,11.0,[Unity],[Single player],[Side view],67.697619,NaN,"[Action, Kids]",[845],NaN,NaN
9,QWOP,2008-11-01,2900,2,29,19,"['Noodlecake Studios Inc.', 'Bennett Foddy']",2.0,"['Indie', 'Sport']","['Android', 'Mobile', 'Web browser']",...,NaN,None,"[Single player, Multiplayer]",[Side view],74.946860,QWOP is the local sporting hero of a prosperou...,"[Action, Comedy]",NaN,NaN,NaN


In [32]:
# data[data['game_id'].isin(missed['game_id'])].sort_values(by='plays')

In [44]:
# You need to ensure the categorical columns can go in as strings
# Because BiqQuery treats it like a JSON
list_of_lists = ['game_modes', 'franchises', 'franchise', 'player_perspectives', 'themes', 'multiplayer_modes', 'game_engines']
everything[list_of_lists] = everything[list_of_lists].astype(str)

#Uploads to the cloud
job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')
job = client.load_table_from_dataframe(everything, 'legendary-game-recs.game_data_01_24.all_data', job_config=job_config)

#Uploads to the cloud
job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')
job = client.load_table_from_dataframe(missed, 'legendary-game-recs.game_data_01_24.missed_api_data', job_config=job_config)


# IGDB and Backlogged Data - KNN

In [67]:
from utils.processing_utils import process_raw_data, multilable_encoding, yeo_johnson_scaling

from sklearn.neighbors import KNeighborsRegressor

import nltk

import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from preprocessing.preprocess_2_features import keep_x_OHE_columns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.preprocessing import MultiLabelBinarizer,PowerTransformer

from sklearn.preprocessing import MinMaxScaler


In [68]:
client = bigquery.Client.from_service_account_json(os.environ.get('JSON_KEY'))

project = os.environ.get('PROJECT')
dataset = os.environ.get('DATASET')
links_table = os.environ.get('LINKS_TABLE')
game_data_table = os.environ.get('GAME_DATA_TABLE')
missed_data_table = os.environ.get('MISSED_DATA_TABLE')

query = f"""
            SELECT DISTINCT * FROM `legendary-game-recs.game_data_01_24.all_data`
        """

query_job = client.query(query)
results = query_job.result()
all_data = results.to_dataframe()


In [69]:
all_data

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,aggregated_rating_count,game_engines,game_modes,player_perspectives,rating,storyline,themes,franchises,multiplayer_modes,franchise
0,Mario Party 10,2015-03-12,2400,4,146,82,['Nintendo'],2.0,"['Card & Board Game', 'Quiz/Trivia']",['Wii U'],...,16.0,None,"['Single player', 'Multiplayer', 'Split screen']",['Third person'],61.310368,None,"['Action', 'Party']",[845],[19938],nan
1,Homefront,2011-03-15,2400,9,404,96,"['Digital Extremes', 'THQ']",2.0,['Shooter'],"['Windows PC', 'Xbox 360', 'PlayStation 3']",...,14.0,['Unreal Engine'],"['Single player', 'Multiplayer']",['First person'],68.646107,The year is 2027. The world has suffered a dec...,"['Action', 'Fantasy', 'Historical', 'Warfare']",nan,nan,nan
2,Flappy Bird,2013-05-24,12000,15,97,17,['.GEARS Studios'],2.0,['Arcade'],"['Android', 'iOS']",...,NaN,None,['Single player'],['Side view'],47.128321,None,['Action'],nan,nan,nan
3,Saints Row,2022-08-23,1300,105,863,383,"['Volition', 'Deep Silver']",2.0,"['Adventure', 'Shooter']","['Windows PC', 'PlayStation 4', 'Xbox One', 'P...",...,7.0,None,"['Single player', 'Multiplayer', 'Co-operative']",['Third person'],46.657578,"As the future Boss, with Neenah, Kevin, and El...","['Action', 'Open world']",nan,[25622],nan
4,Finding Nemo,2003-05-09,1300,5,85,33,"[""Traveller's Tales"", 'THQ']",2.0,['Adventure'],"['Nintendo GameCube', 'Xbox', 'PlayStation 2']",...,3.0,None,['Single player'],['Third person'],60.707378,None,"['Action', 'Educational']",[167],nan,167.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35572,Deal or No Deal,2006-11-06,116,0,10,4,"['Zoo Games', 'Imagination Entertainment']",1.8,['Quiz/Trivia'],"['Windows PC', 'Wii']",...,1.0,None,['Single player'],None,NaN,None,None,[292],nan,292.0
35573,Bro Falls: Ultimate Showdown,2021-06-25,146,3,22,5,[],1.8,"['Indie', 'Racing']",['Windows PC'],...,NaN,['Unity'],None,['Third person'],65.528286,None,['Action'],nan,nan,nan
35574,Blood II: The Chosen,1998-10-31,147,2,79,34,"['Atari, Inc.', 'Monolith Productions']",1.8,['Shooter'],['Windows PC'],...,NaN,['LithTech'],"['Single player', 'Multiplayer']",['First person'],70.862133,None,"['Action', 'Science fiction', 'Horror', 'Comedy']",nan,nan,nan
35575,Assassin's Creed: Rebellion,2018-11-21,98,1,10,3,['Behaviour Interactive'],1.8,['Simulator'],"['Android', 'iOS']",...,NaN,None,['Single player'],['Side view'],59.774895,None,"['Fantasy', 'Historical']",[571],nan,nan


In [115]:
cols_to_remove = ['title', 'release_date', 'plays', 'playing', 'backlogs', 'wishlist',
       'developers', 'avg_review', 'genres', 'platforms', 'description',
       'total_reviews', 'total_lists', 'category', 'main', 'ratings_zero_five',
       'ratings_one_zero', 'ratings_one_five', 'ratings_two_zero',
       'ratings_two_five', 'ratings_three_zero', 'ratings_three_five',
       'ratings_four_zero', 'ratings_four_five', 'ratings_five_zero', 'url',
       'game_id', 'id', 'aggregated_rating', 'aggregated_rating_count',
       'game_engines', 'game_modes', 'player_perspectives', 'rating',
       'storyline', 'themes', 'franchises', 'multiplayer_modes', 'franchise']

cols_to_keep = ['player_perspectives', 'genres', 'description',
                'playing', 'backlogs', 'wishlist', 'avg_review', 'total_reviews', 'total_lists']

num_cols_to_keep = ['plays', 'playing', 'backlogs', 'wishlist', 'avg_review', 'total_reviews', 'total_lists']

cat_cols_to_keep = ['player_perspectives', 'genres']

In [116]:
released_games, upcoming_games = process_raw_data(all_data, 2024, 1, 26, cat_cols_to_keep)
all_games_processed = pd.concat([released_games, upcoming_games])

clean_all_games = all_games_processed[all_games_processed['playing'] >= 0]

In [117]:
model_data = clean_all_games[cols_to_keep].copy()

In [118]:
model_data.isna().sum(), model_data.shape

(player_perspectives    0
 genres                 0
 description            0
 playing                0
 backlogs               0
 wishlist               0
 avg_review             0
 total_reviews          0
 total_lists            0
 dtype: int64,
 (35574, 9))

### NLP

In [119]:
# FUNCTIONS
def preprocess_text(sentence):
    """
    Function to clean description text, tokenize and lemmatize words
    """

    # List of extra stopwords we decided do not help create new genres
    extended_stop_words = ['able','access','across','also','always','another','away',
                        'back','become','best','better','big','box','bring','certain',
                        'clear','close','come','console','content','could','course','digital',
                        'dont','one','two','three','four','five','six','seven','eight','nine',
                        'ten','hundred','thousand','either','enjoy','enough','even','exclusive',
                        'extra','feature','franchise','full','fully','fun','game','gameplay',
                        'genre','get','give','go','good','great','great','greatest','happen',
                        'however','huge','ii','improve','include','increase','inside',
                        'interactive','introduce','instead','involve','know','large','last',
                        'later','launch','lead','let','level','like','little','look','long',
                        'main','may','meet','might','mix','modern','mode','much','must','nan',
                        'name','new','next','need','number','nintendo','official','offer',
                        'object','option','order','original','originally','others','part','pc',
                        'perfect','platform','play','playable','player','playstation','plus',
                        'possible','port','prepare','previous','progress','project','publish',
                        'put','reach','ready','remain','return','screen','scroll','second',
                        'first','third','see','sega','sequel','series','set','several','show',
                        'side','similar','since','small','something','sound','special','start',
                        'stat','state','stay','still','studio','super','take','switch','tell',
                        'test','th','though','throughout','title','together','top','try',
                        'ultimate','unique','update','upon','us','use','version','via','wait',
                        'want','wii','within','without','would','xbox','youll','youre','youve']

    ## define stopwords
    stop_words = stopwords.words('english')
    stop_words.extend(extended_stop_words)

    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

    # Advanced cleaning

    # remove punctuation
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '')

    # tokenize
    tokenized_sentence = word_tokenize(sentence)

    # remove stopwords
    tokenized_sentence_cleaned = [
        w for w in tokenized_sentence if not w in stop_words]


    noun_lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "n")
        for word in tokenized_sentence_cleaned]

    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v")
        for word in noun_lemmatized]

    cleaned_sentence = ' '.join(word for word in lemmatized)

    return cleaned_sentence
def vectorize_text(data, column, min_df=0.02):
    vectorizer = TfidfVectorizer(min_df=min_df)

    # Fit transform on clean text
    vectorized_descriptions = vectorizer.fit_transform(data[column])

    # Create dataframe of vectorized descriptions
    vectorized_descriptions = pd.DataFrame(
        vectorized_descriptions.toarray(),
        columns = vectorizer.get_feature_names_out())

    return vectorizer, vectorized_descriptions
def nlp_topic(x, vectorizer, lda_model):
    """
    Determines which topic the description belongs to
    Should be used to add topic to a column in the training data
    """
    vectorized_x = vectorizer.transform(pd.Series(x))
    vec_x_array = lda_model.transform(vectorized_x)
    vec_list = vec_x_array[0].tolist()

    return vec_list.index(max(vec_list))
def create_nlp_topics_and_append(data, column_to_clean, origin_column, lda_components=30, total_features_to_make=None):
    # Create dataframe of vectorized descriptions
    vectorizer, vectorized_descriptions = vectorize_text(data, column_to_clean)

    # Instantiate LDA model
    lda_model = LatentDirichletAllocation(n_components=lda_components, max_iter = 20, learning_method='online')

    # Fit the LDA on the vectorized documents
    lda_model.fit(vectorized_descriptions)

    # Creating topics column, and encoding into dara
    data['topic'] = data[column_to_clean].apply(nlp_topic, vectorizer = vectorizer, lda_model = lda_model)
    topics = pd.get_dummies(data['topic'])

    if total_features_to_make == None:
        pass
    else:
        topics = keep_x_OHE_columns(topics, total_features_to_make)

    # Creating final df to be split into training data
    concatenated_df = data.drop(columns=[origin_column, column_to_clean, 'topic'], axis=1)
    final_df = pd.concat((concatenated_df, topics), axis=1)

    return final_df


def topics_from_nlp(data, column_with_text, number_of_topics_to_keep=None, lda_components=30):

    data['clean_text'] = data[column_with_text].apply(preprocess_text)
    topics_nlp_df = create_nlp_topics_and_append(data, column_to_clean='clean_text',
                                                 origin_column=column_with_text, total_features_to_make=number_of_topics_to_keep,
                                                 lda_components=lda_components)

    return topics_nlp_df
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
cols_rename_dict = {0:'topic_0', 1:'topic_1',2:'topic_2',3:'topic_3',4:'topic_4',5:'topic_5',6:'topic_6',7:'topic_7',8:'topic_8',9:'topic_9',10:'topic_10',
                    11:'topic_11',12:'topic_12',13:'topic_13',14:'topic_14',15:'topic_15',16:'topic_16',17:'topic17',18:'topic_18',19:'topic_19',20:'topic_20',
                    21:'topic_21',22:'topic_22',23:'topic_23',24:'topic_24',25:'topic_25',26:'topic_26',27:'topic_27',28:'topic_28',29:'topic_29',30:'topic_30'}
def predict_baseline_model(indicies, model, reference_data, X_train):
    game = X_train[indicies:indicies+1]

    ind_list = list(model.kneighbors(game,n_neighbors=10)[1][0])
    prediction = reference_data.iloc[ind_list]

    return prediction

In [120]:
# NLP SECTION

model_data['description'] = model_data['description'].astype(str)
model_data['clean_text'] = model_data['description'].apply(preprocess_text)


topics_nlp_df = create_nlp_topics_and_append(model_data, column_to_clean='clean_text', origin_column='description', lda_components=27)
topics_nlp_df.columns

topics_nlp_df.rename(columns=cols_rename_dict, inplace=True)


/Users/antonis/.pyenv/versions/3.11.2/envs/game_recs_env/lib/python3.11/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LatentDirichletAllocation was fitted with feature names
  warnings.warn(


In [121]:
topics_nlp_df.shape

(35574, 35)

In [122]:
genre_list = ['Adventure', 'Indie', 'RPG', 'Simulator', 'Puzzle',
 'Shooter', 'Platform', 'Arcade', 'Sport', 'Racing',
 'Fighting', 'Turn Based Strategy', 'Visual Novel']

In [123]:
def multilable_encoding_fix2(data, column, len_of_feat=11):
    """
    This ohe encodes categories that are stored in lists
    """
    # Initilizing encoder and transforming the data
    mlb_genre = MultiLabelBinarizer()
    transformed_genre = mlb_genre.fit_transform(data[column])

    # Labeling newly created features
    genre_ohe_colums = pd.DataFrame(transformed_genre, columns=mlb_genre.classes_)
    # Concatinating the data

    droped_columns = data.drop(column, axis=1).copy()

    list_of_feats = pd.DataFrame(genre_ohe_colums.sum().sort_values(ascending=False)[:len_of_feat]).index.to_list()

    scaled_data = pd.concat((droped_columns, genre_ohe_colums[list_of_feats]), axis=1)

    scaled_data.rename(columns={'':f'NaN_{column}',}, inplace=True)

    return scaled_data

In [124]:
genres_encoded = multilable_encoding_fix2(topics_nlp_df, 'genres', len_of_feat=10)
perspectives_and_genres_encoded = multilable_encoding_fix2(genres_encoded, 'player_perspectives')

TypeError: 'float' object is not iterable

In [125]:
genres_encoded.isna().sum()

player_perspectives    3
playing                3
backlogs               3
wishlist               3
avg_review             3
total_reviews          3
total_lists            3
topic_0                3
topic_1                3
topic_2                3
topic_3                3
topic_4                3
topic_5                3
topic_6                3
topic_7                3
topic_8                3
topic_9                3
topic_10               3
topic_11               3
topic_12               3
topic_13               3
topic_14               3
topic_15               3
topic_16               3
topic17                3
topic_18               3
topic_19               3
topic_20               3
topic_21               3
topic_22               3
topic_23               3
topic_24               3
topic_25               3
topic_26               3
Adventure              3
Indie                  3
RPG                    3
Simulator              3
Puzzle                 3
Strategy               3


In [ ]:
perspectives_and_genres_encoded

,plays,playing,backlogs,wishlist,avg_review,total_reviews,total_lists,topic_0,topic_1,topic_2,...,Arcade,Sport,NaN_player_perspectives,Third person,Bird view / Isometric,Side view,First person,Text,Virtual Reality,Auditory
0,2400,4,146,82,2.0,132,285,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2400,9,404,96,2.0,83,174,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,12000,15,97,17,2.0,349,218,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1300,105,863,383,2.0,240,344,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1300,5,85,33,2.0,58,114,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18403,20,2,13,25,3.3,2,18,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22290,11,0,2,3,2.2,1,4,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28657,19,0,36,210,2.1,12,32,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33266,29,2,80,206,3.6,3,76,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
lists = yeo_johnson_scaling(perspectives_and_genres_encoded, 'total_lists')
plays = yeo_johnson_scaling(lists, 'plays')
reviews = yeo_johnson_scaling(plays, 'total_reviews')
playing = yeo_johnson_scaling(reviews, 'playing')
wishlist = yeo_johnson_scaling(playing, 'wishlist')
backlogs = yeo_johnson_scaling(wishlist, 'backlogs')

In [103]:
# avg_scaler = MinMaxScaler()

# scaled_and_encoded_data['avg_review'] = avg_scaler.fit_transform(scaled_and_encoded_data[['avg_review']]).squeeze()

In [104]:
baseline_data = backlogs.copy()

In [114]:
baseline_data.columns

Index(['plays', 'playing', 'backlogs', 'wishlist', 'avg_review',
       'total_reviews', 'total_lists', 'topic_0', 'topic_1', 'topic_2',
       'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8',
       'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic_13', 'topic_14',
       'topic_15', 'topic_16', 'topic17', 'topic_18', 'topic_19', 'topic_20',
       'topic_21', 'topic_22', 'topic_23', 'topic_24', 'topic_25', 'topic_26',
       'Adventure', 'Indie', 'RPG', 'Simulator', 'Puzzle', 'Strategy',
       'Shooter', 'Platform', 'Arcade', 'Sport', 'NaN_player_perspectives',
       'Third person', 'Bird view / Isometric', 'Side view', 'First person',
       'Text', 'Virtual Reality', 'Auditory'],
      dtype='object')

In [107]:
baseline_data.columns

Index(['plays', 'playing', 'backlogs', 'wishlist', 'avg_review',
       'total_reviews', 'total_lists', 'topic_0', 'topic_1', 'topic_2',
       'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8',
       'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic_13', 'topic_14',
       'topic_15', 'topic_16', 'topic17', 'topic_18', 'topic_19', 'topic_20',
       'topic_21', 'topic_22', 'topic_23', 'topic_24', 'topic_25', 'topic_26',
       'Adventure', 'Indie', 'RPG', 'Simulator', 'Puzzle', 'Strategy',
       'Shooter', 'Platform', 'Arcade', 'Sport', 'NaN_player_perspectives',
       'Third person', 'Bird view / Isometric', 'Side view', 'First person',
       'Text', 'Virtual Reality', 'Auditory'],
      dtype='object')

In [108]:
X_train_baseline = baseline_data.drop('avg_review', axis=1)
y_train_baseline = baseline_data['avg_review']

baseline_knn = KNeighborsRegressor(n_neighbors=11).fit(X_train_baseline,
                                                       y_train_baseline)

# Extra params for KNN - weights='distance', metric='euclidean'

In [109]:
all_games_processed.reset_index(drop=True, inplace=True)
baseline_data.reset_index(drop=True, inplace=True)

In [110]:
all_games_processed.shape, baseline_data.shape

((35577, 39), (35568, 52))

In [111]:
all_games_processed[(all_games_processed['title'] == 'Call of Duty: Black Ops 4')
               | (all_games_processed['title'] == 'The Legend of Zelda: Breath of the Wild')
              | (all_games_processed['title'] == 'Omori')
              | (all_games_processed['title'] == 'FIFA 21')
              | (all_games_processed['title'] == 'Tetris')
              | (all_games_processed['title'] == 'Sid Meier\'s Civilization V')
              | (all_games_processed['title'] == 'Red Dead Redemption')
              | (all_games_processed['title'] == 'The Sims 4')
              | (all_games_processed['title'] == 'Animal Crossing: New Horizons')
              | (all_games_processed['title'] == 'Immortals Fenyx Rising')]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,aggregated_rating_count,game_engines,game_modes,player_perspectives,rating,storyline,themes,franchises,multiplayer_modes,franchise
170,Tetris,2004-12-31,5,0,0,1,[],2.0,[Puzzle],['Fairchild Channel F'],...,NaN,None,None,[],NaN,None,None,nan,nan,nan
800,FIFA 21,2020-10-08,1900,37,69,21,"['Electronic Arts', 'EA Vancouver']",2.5,"[Simulator, Sport]","['Windows PC', 'PlayStation 4', 'Xbox One', 'P...",...,12.0,['Frostbite'],"['Single player', 'Multiplayer', 'Co-operative...","[Third person, Bird view / Isometric]",69.197008,None,['Non-fiction'],[1110],nan,nan
6120,Sid Meier's Civilization V,2010-09-21,8600,84,572,201,"['2K Games', 'Firaxis Games']",4.0,"[Simulator, Strategy, Turn Based Strategy]","['Windows PC', 'Mac', 'Linux']",...,9.0,['Firaxis LORE'],"['Single player', 'Multiplayer']",[Bird view / Isometric],85.832256,None,"['Fantasy', 'Science fiction', 'Historical', '...",[10],nan,10.0
6681,Red Dead Redemption,2023-08-17,94,13,46,47,"['Double Eleven', 'Rockstar Games']",4.0,"[Adventure, RPG, Shooter]","['PlayStation 4', 'Nintendo Switch']",...,NaN,None,['Single player'],[Third person],NaN,None,"['Action', 'Sandbox', 'Open world']",nan,nan,nan
14176,Call of Duty: Black Ops 4,2018-10-12,6400,33,501,185,"['Activision', 'Treyarch']",2.3,[Shooter],"['Windows PC', 'PlayStation 4', 'Xbox One']",...,20.0,['IW Engine 3.0'],"['Single player', 'Multiplayer', 'Co-operative...",[First person],67.157262,None,"['Action', 'Warfare']",[726],[21232],nan
20350,Omori,2020-12-25,14000,1500,8200,6100,"['PLAYISM', 'OMOCAT']",4.1,"[Adventure, Indie, RPG, Turn Based Strategy]","['Windows PC', 'Mac', 'PlayStation 4', 'Ninten...",...,2.0,['RPG Maker MV'],['Single player'],[Bird view / Isometric],89.088822,"Welcome to WHITE SPACE.\nYour name is OMORI, a...","['Fantasy', 'Horror']",nan,nan,nan
24312,Immortals Fenyx Rising,2020-12-02,2800,205,1700,636,"['Ubisoft Entertainment', 'Ubisoft Québec']",3.2,"[Adventure, Puzzle, RPG]","['Windows PC', 'PlayStation 4', 'Xbox One', 'P...",...,13.0,['Ubisoft Anvil'],['Single player'],[Third person],77.009794,None,"['Action', 'Fantasy']",nan,nan,nan
27771,The Legend of Zelda: Breath of the Wild,2017-03-03,53000,4000,8400,4200,"['Nintendo EPD Production Group No. 3', 'Ninte...",4.4,"[Adventure, Puzzle, RPG]","['Wii U', 'Nintendo Switch']",...,31.0,"['Havok Physics', 'In-house engine']",['Single player'],[Third person],92.800736,Link is awakened in a room by a voice calling ...,"['Action', 'Fantasy', 'Science fiction', 'Surv...",[596],nan,nan
30831,The Sims 4,2014-09-02,16000,816,1300,163,"['Electronic Arts', 'The Sims Studio']",3.1,"[RPG, Simulator]","['Windows PC', 'Mac', 'PlayStation 4', 'Xbox O...",...,16.0,['SmartSim'],['Single player'],"[First person, Third person, Bird view / Isome...",73.464612,"Choose how Sims look, act, and dress. Determin...","['Action', 'Fantasy', 'Comedy', 'Sandbox', 'Ro...",[979],nan,nan
32758,Animal Crossing: New Horizons,2020-03-19,33000,2400,2400,1800,"['Nintendo EPD', 'Nintendo']",3.6,[Simulator],['Nintendo Switch'],...,15.0,None,"['Single player', 'Multiplayer', 'Co-operative...",[Bird view / Isometric],82.569688,If the hustle and bustle of modern life’s got ...,"['Sandbox', 'Kids']",[1820],[9587],nan


20332 = Slay the Spire

In [113]:
predict_baseline_model(800, baseline_knn, all_games_processed, X_train_baseline)

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,aggregated_rating_count,game_engines,game_modes,player_perspectives,rating,storyline,themes,franchises,multiplayer_modes,franchise
800,FIFA 21,2020-10-08,1900,37,69,21,"['Electronic Arts', 'EA Vancouver']",2.5,"[Simulator, Sport]","['Windows PC', 'PlayStation 4', 'Xbox One', 'P...",...,12.0,['Frostbite'],"['Single player', 'Multiplayer', 'Co-operative...","[Third person, Bird view / Isometric]",69.197008,None,['Non-fiction'],[1110],nan,nan
22893,P.N.03,2003-03-27,302,6,356,214,"['Capcom', 'Capcom Production Studio 4']",2.7,[Shooter],['Nintendo GameCube'],...,1.0,None,['Single player'],[Third person],63.361976,None,"['Action', 'Science fiction']",nan,nan,nan
29297,Smite,2014-03-25,4800,104,224,23,"['Tencent Games', 'Titan Forge Games']",2.6,"[Adventure, MOBA, RPG, Strategy]","['Windows PC', 'Mac', 'PlayStation 4', 'Xbox O...",...,6.0,['Unreal Engine'],"['Multiplayer', 'Co-operative']",[Third person],69.383228,None,"['Action', 'Fantasy']",nan,"[8259, 20894]",nan
874,Jump Rope Challenge,2020-06-15,576,5,106,6,['Nintendo'],2.5,[Sport],['Nintendo Switch'],...,NaN,['Unity'],"['Single player', 'Multiplayer']",[Third person],53.603415,None,None,nan,[11054],nan
24417,LEGO Island,1997-09-26,933,0,105,94,['Mindscape'],3.2,[Adventure],['Windows PC'],...,NaN,None,['Single player'],[First person],67.590144,Pepper gets a call from a criminal in jail cal...,"['Action', 'Comedy', 'Kids', 'Open world']",[51],nan,51.0
27613,Picross DS,2007-01-25,474,20,84,37,"['Nintendo', 'Jupiter Corporation']",3.7,[Puzzle],['Nintendo DS'],...,2.0,None,['Single player'],[Text],64.013497,None,None,nan,nan,nan
33149,Verne: The Shape of Fantasy,2023-08-14,13,1,32,34,['Gametopia Studios'],3.6,"[Adventure, Indie, Puzzle]","['Windows PC', 'Mac']",...,NaN,['Unity'],['Single player'],[Side view],NaN,Jules Verne has been trapped in parallel world...,None,nan,nan,nan
801,FIFA 19,2018-09-28,2300,15,84,11,"['EA Sports', 'EA Vancouver']",2.5,[Sport],"['Windows PC', 'PlayStation 4', 'Xbox One']",...,13.0,['Frostbite'],"['Single player', 'Multiplayer', 'Co-operative']","[Third person, Bird view / Isometric]",73.179531,None,['Non-fiction'],[1110],nan,nan
8867,Ben 10 Ultimate Alien: Cosmic Destruction,2010-10-05,1600,4,104,56,"['Griptonite Games', 'D3 Publisher']",2.9,[Adventure],"['Xbox 360', 'PlayStation 3', 'Nintendo DS', '...",...,3.0,None,['Single player'],[Third person],65.807061,The game involves Ben traveling to iconic real...,"['Action', 'Science fiction']",[101],nan,101.0
12121,Convergence: A League of Legends Story,2023-05-23,188,12,103,181,"['Riot Forge', 'Double Stallion Games']",3.4,"[Adventure, Indie, Platform]","['Windows PC', 'PlayStation 4', 'Xbox One', 'P...",...,3.0,None,['Single player'],[],60.545620,None,['Action'],[1572],nan,nan


In [568]:
all_games_processed[all_games_processed['title'] == 'Immortals Fenyx Rising']

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,aggregated_rating_count,game_engines,game_modes,player_perspectives,rating,storyline,themes,franchises,multiplayer_modes,franchise
24319,Immortals Fenyx Rising,2020-12-02,2800,205,1700,636,"['Ubisoft Entertainment', 'Ubisoft Québec']",3.2,"[Adventure, Puzzle, RPG]","['Windows PC', 'PlayStation 4', 'Xbox One', 'P...",...,13.0,['Ubisoft Anvil'],['Single player'],[Third person],77.009794,None,"['Action', 'Fantasy']",nan,nan,nan


# GraphDatabase

In [7]:
from utils.processing_utils import process_raw_data, multilable_encoding, yeo_johnson_scaling

from sklearn.neighbors import KNeighborsRegressor


In [8]:
client = bigquery.Client.from_service_account_json(os.environ.get('JSON_KEY'))

project = os.environ.get('PROJECT')
dataset = os.environ.get('DATASET')
links_table = os.environ.get('LINKS_TABLE')
game_data_table = os.environ.get('GAME_DATA_TABLE')
missed_data_table = os.environ.get('MISSED_DATA_TABLE')

query = f"""
            SELECT DISTINCT * FROM `legendary-game-recs.game_data_01_24.all_data`
        """

query_job = client.query(query)
results = query_job.result()
all_data = results.to_dataframe()

In [9]:
all_data.columns

Index(['title', 'release_date', 'plays', 'playing', 'backlogs', 'wishlist',
       'developers', 'avg_review', 'genres', 'platforms', 'description',
       'total_reviews', 'total_lists', 'category', 'main', 'ratings_zero_five',
       'ratings_one_zero', 'ratings_one_five', 'ratings_two_zero',
       'ratings_two_five', 'ratings_three_zero', 'ratings_three_five',
       'ratings_four_zero', 'ratings_four_five', 'ratings_five_zero', 'url',
       'game_id', 'id', 'aggregated_rating', 'aggregated_rating_count',
       'game_engines', 'game_modes', 'player_perspectives', 'rating',
       'storyline', 'themes', 'franchises', 'multiplayer_modes', 'franchise'],
      dtype='object')

In [10]:
columns_to_keep = ['developers', 'genres', 'platforms',
       'game_engines', 'game_modes', 'player_perspectives', 'themes', 'franchises', 'multiplayer_modes',

       'avg_review', 'title', 'ratings_one_zero', 'backlogs', 'total_reviews',
             'ratings_two_zero', 'ratings_one_five', 'storyline', 'total_lists', 'ratings_four_zero',
             'ratings_two_five','playing', 'ratings_three_zero', 'description', 'ratings_four_five', 'release_date', 'plays',
             'ratings_five_zero', 'url', 'ratings_zero_five', 'ratings_three_five', 'wishlist', 'game_id']

relationship_cols = ['developers', 'genres', 'platforms', 'game_id',
       'game_engines', 'game_modes', 'player_perspectives', 'themes', 'franchises', 'multiplayer_modes']

game_data = ['avg_review', 'title', 'ratings_one_zero', 'backlogs', 'total_reviews',
             'ratings_two_zero', 'ratings_one_five', 'storyline', 'total_lists', 'ratings_four_zero',
             'ratings_two_five','playing', 'ratings_three_zero', 'description', 'ratings_four_five', 'release_date', 'plays',
             'ratings_five_zero', 'url', 'ratings_zero_five', 'ratings_three_five', 'wishlist', 'game_id']

cat_list_columns = ['developers', 'genres', 'platforms','game_engines', 'game_modes', 'player_perspectives',
                    'themes', 'franchises', 'multiplayer_modes']

In [11]:
all_data[relationship_cols]

,developers,genres,platforms,game_id,game_engines,game_modes,player_perspectives,themes,franchises,multiplayer_modes
0,['Nintendo'],"['Card & Board Game', 'Quiz/Trivia']",['Wii U'],7340,None,"['Single player', 'Multiplayer', 'Split screen']",['Third person'],"['Action', 'Party']",[845],[19938]
1,"['Digital Extremes', 'THQ']",['Shooter'],"['Windows PC', 'Xbox 360', 'PlayStation 3']",1885,['Unreal Engine'],"['Single player', 'Multiplayer']",['First person'],"['Action', 'Fantasy', 'Historical', 'Warfare']",nan,nan
2,['.GEARS Studios'],['Arcade'],"['Android', 'iOS']",7671,None,['Single player'],['Side view'],['Action'],nan,nan
3,"['Volition', 'Deep Silver']","['Adventure', 'Shooter']","['Windows PC', 'PlayStation 4', 'Xbox One', 'P...",165346,None,"['Single player', 'Multiplayer', 'Co-operative']",['Third person'],"['Action', 'Open world']",nan,[25622]
4,"[""Traveller's Tales"", 'THQ']",['Adventure'],"['Nintendo GameCube', 'Xbox', 'PlayStation 2']",3781,None,['Single player'],['Third person'],"['Action', 'Educational']",[167],nan
...,...,...,...,...,...,...,...,...,...,...
35572,"['Zoo Games', 'Imagination Entertainment']",['Quiz/Trivia'],"['Windows PC', 'Wii']",4800,None,['Single player'],None,None,[292],nan
35573,[],"['Indie', 'Racing']",['Windows PC'],153636,['Unity'],None,['Third person'],['Action'],nan,nan
35574,"['Atari, Inc.', 'Monolith Productions']",['Shooter'],['Windows PC'],11265,['LithTech'],"['Single player', 'Multiplayer']",['First person'],"['Action', 'Science fiction', 'Horror', 'Comedy']",nan,nan
35575,['Behaviour Interactive'],['Simulator'],"['Android', 'iOS']",41030,None,['Single player'],['Side view'],"['Fantasy', 'Historical']",[571],nan


In [12]:
released_games, upcoming_games = process_raw_data(all_data, 2024, 1, 26, cat_list_columns)
all_games_processed = pd.concat([released_games, upcoming_games])
model_data = all_games_processed[columns_to_keep].copy()

In [13]:
graph_game_data = model_data[game_data].copy()

In [14]:
graph_relationship_data  = model_data[relationship_cols].copy()

In [15]:
graph_relationship_data

,developers,genres,platforms,game_id,game_engines,game_modes,player_perspectives,themes,franchises,multiplayer_modes
0,[Nintendo],"[Card & Board Game, Quiz/Trivia]",[Wii U],7340,[],"[Single player, Multiplayer, Split screen]",[Third person],"[Action, Party]",[4],[993]
1,"[Digital Extremes, THQ]",[Shooter],"[Windows PC, Xbox 360, PlayStation 3]",1885,[Unreal Engine],"[Single player, Multiplayer]",[First person],"[Action, Fantasy, Historical, Warfare]",[],[]
2,[.GEARS Studios],[Arcade],"[Android, iOS]",7671,[],[Single player],[Side view],[Action],[],[]
3,"[Volition, Deep Silver]","[Adventure, Shooter]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",165346,[],"[Single player, Multiplayer, Co-operative]",[Third person],"[Action, Open world]",[],[562]
4,"[Travellers Tales"", THQ]",[Adventure],"[Nintendo GameCube, Xbox, PlayStation 2]",3781,[],[Single player],[Third person],"[Action, Educational]",[6],[]
...,...,...,...,...,...,...,...,...,...,...
18403,[DitD Team],"[Card & Board Game, Indie, RPG, Strategy]","[Windows PC, Mac, Linux]",147285,[Unity],[Single player],[Side view],[Fantasy],[],[]
22290,[Melancholy Marionette],[Visual Novel],"[Windows PC, Mac, Linux]",178780,[Unity],[Single player],[Text],[Romance],[],[]
28657,"[Nuuvem, Dumativa Game Studio]","[Adventure, Indie, Puzzle]",[Windows PC],140133,[Unity],[Single player],[Bird view / Isometric],"[Action, Horror, Survival]",[],[]
33266,[],"[Adventure, Shooter]","[Windows PC, Linux]",145828,[GZDoom],[],[First person],[Action],[],[]


In [25]:
# Select only the columns that contain relationships
temp_rel_cols = ['player_perspectives']

# Create an empty list to store relationship data
relationship_data = []

for index, row in graph_relationship_data.iterrows():
    game_id = row['game_id']
    for col in temp_rel_cols:
        # Extract relationships from the list
        relationships = row[col]
        # Create a row for each relationship
        for relationship in relationships:
            relationship_data.append({'game_id': game_id, 'attribute': col, 'value': relationship})

# Create a DataFrame from the relationship data
relationship_df = pd.DataFrame(relationship_data)

In [26]:
relationship_df

,game_id,attribute,value
0,7340,player_perspectives,Third person
1,1885,player_perspectives,First person
2,7671,player_perspectives,Side view
3,165346,player_perspectives,Third person
4,3781,player_perspectives,Third person
...,...,...,...
39195,147285,player_perspectives,Side view
39196,178780,player_perspectives,Text
39197,140133,player_perspectives,Bird view / Isometric
39198,145828,player_perspectives,First person


In [27]:
relationship_df.to_csv('play_perspective_relationships_test.csv', index=False)
# graph_game_data.to_csv('game_data_test.csv', index=False)